In [19]:
!pip3 install -q transformers


[notice] A new release of pip is available: 23.0 -> 23.0.1
[notice] To update, run: pip3 install --upgrade pip


In [20]:
import numpy as np
import pandas as pd
from sklearn import metrics
import transformers
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertModel, BertConfig, AutoTokenizer, AutoModelForMaskedLM, AutoModelForSequenceClassification, AutoConfig

In [21]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [22]:
from google.colab import drive
drive.mount('/content/gdrive')

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
%cd /content/gdrive/MyDrive/CS224NFinalProj/

[Errno 2] No such file or directory: '/content/gdrive/MyDrive/CS224NFinalProj/'
/content


In [23]:
df = pd.read_csv("patient_data.csv")
#df['list'] = df[df.columns[1:3]].values.tolist()
new_df = df[['id', 'doctor_faculty', 'description']].copy()
new_df.head()

,id,doctor_faculty,description
0,8,妇产科,疾病： 孕期不舒服病情描述： 近两天肚子有下坠感，不痛，心跳比较快，其他没有不舒服希望获得的...
1,9,皮肤科,疾病： 指甲上突起的横纹病情描述： 上周发现拇指指甲上突起一道横纹！不疼不痒，突起来的横纹！...
2,10,妇科,疾病： hpv疫苗病情描述： 我每两年做一次宫颈癌全套筛查都是阴性，无任何感染。tct阴，去...
3,11,眼科,疾病： 宝宝眼角红红的，严重时轻微溃烂。病情描述： 宝宝眼角红红的氧，用小手挠，严重时轻微溃...
4,13,皮肤科,疾病： 宝宝奶粉过敏、湿疹病情描述： 两个礼拜前喂了普通的飞鹤奶粉，是用勺子喂的，喝完一点点...


In [24]:
#map each department to an index 
departments = new_df['doctor_faculty'].unique()
d2ind = {departments[i]:i for i in range(len(departments))}
ind2d = {i:departments[i] for i in range(len(departments))}

In [25]:
ground_truths = []
for i, row in new_df.iterrows():
  dept_name = row['doctor_faculty']
  dept_ind = d2ind[dept_name]
  one_hot = np.zeros(len(departments))
  one_hot[dept_ind] = 1
  ground_truths.append(one_hot)
new_df['one_hot'] = list(ground_truths)

In [26]:
new_df.drop(['id', 'doctor_faculty'], axis='columns', inplace=True)
new_df.head()

,description,one_hot
0,疾病： 孕期不舒服病情描述： 近两天肚子有下坠感，不痛，心跳比较快，其他没有不舒服希望获得的...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,疾病： 指甲上突起的横纹病情描述： 上周发现拇指指甲上突起一道横纹！不疼不痒，突起来的横纹！...,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,疾病： hpv疫苗病情描述： 我每两年做一次宫颈癌全套筛查都是阴性，无任何感染。tct阴，去...,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,疾病： 宝宝眼角红红的，严重时轻微溃烂。病情描述： 宝宝眼角红红的氧，用小手挠，严重时轻微溃...,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,疾病： 宝宝奶粉过敏、湿疹病情描述： 两个礼拜前喂了普通的飞鹤奶粉，是用勺子喂的，喝完一点点...,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [28]:
## Sections of config

# Defining some key variables that will be used later on in the training
MAX_LEN = 200
TRAIN_BATCH_SIZE = 8
VALID_BATCH_SIZE = 4
EPOCHS = 5
LEARNING_RATE = 1e-05
# load in pre-trained chinese bert
# experiment with chinese-bert-wwm
#tokenizer = BertTokenizer.from_pretrained('uer/chinese_roberta_L-8_H-512')
#tokenizer = AutoTokenizer.from_pretrained('uer/roberta-base-finetuned-chinanews-chinese')
#tokenizer = AutoTokenizer.from_pretrained("uer/roberta-base-finetuned-chinanews-chinese")
tokenizer = AutoTokenizer.from_pretrained("uer/roberta-base-finetuned-chinanews-chinese")

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/jackyuan/Library/Python/3.9/lib/python/site-packages/tqdm/_tqdm_notebook.py", line 103, in status_printer
NameError: name 'IntProgress' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/jackyuan/Library/Python/3.9/lib/python/site-packages/IPython/core/interactiveshell.py", line 3460, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/z3/rlvk8dq50nqf7wvl5mz7sdtr0000gn/T/ipykernel_48728/895140328.py", line 14, in <module>
    tokenizer = AutoTokenizer.from_pretrained("uer/roberta-base-finetuned-chinanews-chinese")
  File "/Users/jackyuan/Library/Python/3.9/lib/python/site-packages/transformers/models/auto/tokenization_auto.py", line 598, in from_pretrained
    tokenizer_config = get_tokenizer_config(pretrained_model_name_or_path, **kwargs)
  File "/Users/jackyuan/Library/Python/3.9/lib/python/site-packages/transformers/

In [ ]:
class CustomDataset(Dataset):

    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.description = dataframe.description
        self.targets = self.data.one_hot
        self.max_len = max_len

    def __len__(self):
        return len(self.description)

    def __getitem__(self, index):
        description = str(self.description[index])
        description = " ".join(description.split())

        inputs = self.tokenizer.encode_plus(
            description,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [ ]:
# Creating the dataset and dataloader for the neural network

train_size = 0.8
train_dataset=new_df.sample(frac=train_size,random_state=200)

In [ ]:
test_dataset=new_df.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)


print("FULL Dataset: {}".format(new_df.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))

training_set = CustomDataset(train_dataset, tokenizer, MAX_LEN)
testing_set = CustomDataset(test_dataset, tokenizer, MAX_LEN)

FULL Dataset: (25930, 2)
TRAIN Dataset: (20744, 2)
TEST Dataset: (5186, 2)


NameError: name 'tokenizer' is not defined

In [ ]:
testing_set[1]

In [ ]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

##Training

In [ ]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("uer/roberta-base-finetuned-chinanews-chinese", num_labels=10, ignore_mismatched_sizes=True, id2label=ind2d)
model.to(device)
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)
def train(epoch):
    model.train()
    for _,data in enumerate(training_loader, 0):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        outputs = model(ids, mask, token_type_ids, labels=targets)
        #sigmoids = torch.nn.functional.sigmoid(outputs.logits)
        #sigmoids = torch.zeros_like(sigmoids).scatter_(1, torch.multinomial(sigmoids,1), 1.)

        optimizer.zero_grad()
        #loss = loss_fn(outputs.logits, targets)

        loss, logits = outputs[:2]

        if _%5000==0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')
        
        optimizer.zero_grad()
        #loss.backward()
        #optimizer.step()
 
        loss.backward()
        optimizer.step()

In [ ]:
for epoch in range(EPOCHS):
    train(epoch)

##Validation

In [ ]:
def validation(epoch):
    model.eval()
    fin_targets=[]
    fin_outputs=[]
    with torch.no_grad():
        for _, data in enumerate(testing_loader, 0):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            outputs = model(ids, mask, token_type_ids)
            sigmoids = torch.sigmoid(outputs.logits).cpu().detach().numpy().tolist()
            idx = np.argmax(sigmoids, axis=-1)
            sigmoids = np.zeros( (len(sigmoids), len(sigmoids[1])) )
            sigmoids[ np.arange(len(sigmoids)), idx] = 1
            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(sigmoids)
    return fin_outputs, fin_targets

In [ ]:
EPOCHS=1
for epoch in range(EPOCHS):
    outputs, targets = validation(epoch)
    print('outputs', outputs)
    print('targets', targets)
    outputs = np.array(outputs) >= 0.5
    accuracy = metrics.accuracy_score(targets, outputs)
    f1_score_micro = metrics.f1_score(targets, outputs, average='micro')
    f1_score_macro = metrics.f1_score(targets, outputs, average='macro')
    print(f"Accuracy Score = {accuracy}")
    print(f"F1 Score (Micro) = {f1_score_micro}")
    print(f"F1 Score (Macro) = {f1_score_macro}")

In [ ]:
torch.save(model.state_dict(), "./roberta-base.bin")

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("uer/roberta-base-finetuned-chinanews-chinese", num_labels=10, ignore_mismatched_sizes=True)
model.to(device)
my_config = AutoConfig.from_pretrained("uer/roberta-base-finetuned-chinanews-chinese", num_labels=10)
print(my_config)
for _,data in enumerate(training_loader, 0):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        outputs = model(ids, mask, token_type_ids)
        predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
        print(predictions.shape) # 8,10


In [ ]:
# Creating the customized model, by adding a drop out and a dense layer on top of distil bert to get the final output for the model. 

class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        #self.l1 = AutoModelForSequenceClassification.from_pretrained("uer/roberta-base-finetuned-chinanews-chinese")
        self.l1 = AutoModelForSequenceClassification.from_pretrained("uer/roberta-base-finetuned-chinanews-chinese", num_labels=10, ignore_mismatched_sizes=True)

        #self.l2 = torch.nn.Dropout(0.3)
        #self.l3 = torch.nn.Linear(21128, 10)
    
    def forward(self, ids, mask, token_type_ids):
        output_1= self.l1(ids, attention_mask = mask, token_type_ids = token_type_ids, return_dict=False)
        
        #print('output_1', output_1[0].shape)
        #output_2 = self.l2(output_1[0])
        #print('output_2', output_2[0].shape)
        #output = self.l3(output_2)
        return output_1.logits

model = BERTClass()
model.to(device)